In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, udf
from pyspark.sql.types import StringType

# Crear sesión de Spark
spark = SparkSession.builder.appName("potencial_jugadores_FIFA").getOrCreate()

# Cargar datos
jugadores_df = spark.read.option("delimiter", ";").csv('player.csv', header=True, inferSchema=True)
atributos_df = spark.read.option("delimiter", ";").csv('player_attributes.csv', header=True, inferSchema=True)

# Realizar join entre DataFrames
join = jugadores_df.select('player_api_id','player_name').join(atributos_df,on='player_api_id',how='left')

# Filtrar valores en blanco o nulos en overall_rating
filtered_join = join.filter(join.overall_rating.isNotNull())

# Calcular la media del potencial (overall_rating)
potencial = join.agg(avg(col("overall_rating"))).collect()[0][0]

# Definir la función para el potencial
def potencial_f(rating):
    return "ALTO" if rating > potencial else "BAJO"

# Registrar la función como una UDF
potencial_udf = udf(potencial_f, StringType())

# Metemos la columna con la función al dataframe
datos_con_potencial_df = join.withColumn("potencial_jugador", potencial_udf(join["overall_rating"]))

# Mostrar el DataFrame con la columna para ver el potencial
# con el ID, el nombre, el potencial y la clasificación según la media del potencial
datos_con_potencial_df.select("player_api_id", "player_name", "overall_rating", "potencial_jugador").show(50)

spark.stop()

+-------------+------------------+--------------+-----------------+
|player_api_id|       player_name|overall_rating|potencial_jugador|
+-------------+------------------+--------------+-----------------+
|       505942|Aaron Appindangoye|            61|             BAJO|
|       505942|Aaron Appindangoye|            61|             BAJO|
|       505942|Aaron Appindangoye|            62|             BAJO|
|       505942|Aaron Appindangoye|            67|             BAJO|
|       505942|Aaron Appindangoye|            67|             BAJO|
|       155782|   Aaron Cresswell|            53|             BAJO|
|       155782|   Aaron Cresswell|            53|             BAJO|
|       155782|   Aaron Cresswell|            47|             BAJO|
|       155782|   Aaron Cresswell|            52|             BAJO|
|       155782|   Aaron Cresswell|            51|             BAJO|
|       155782|   Aaron Cresswell|            54|             BAJO|
|       155782|   Aaron Cresswell|            64